<a href="https://colab.research.google.com/github/albert-yue/gcn-explanability/blob/master/gcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%cd gdrive/My\ Drive/gcn_explainability

/content/gdrive/My Drive/gcn_explainability


In [8]:
! git commit -m "Update"

[master 6d4a833] Update
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite gcn.ipynb (97%)


In [9]:
! git pull

Auto-merging gcn.ipynb
CONFLICT (content): Merge conflict in gcn.ipynb
Automatic merge failed; fix conflicts and then commit the result.


In [13]:
%ls

data/  gcn.ipynb  logs.pt  notebooks/  README.md  src/


In [0]:
from src.models.gcn import GCN
import torch
import numpy as np
import scipy.sparse as sparse
import scipy
from src.data import Corpus, get_data, get_labels, get_vocabulary
from src.utils import load_sparse_tensor
import torch.nn as nn
from torch.autograd import Variable
from src.preprocessing import build_adj_matrix, normalize_adj

logs = (200, 0.5, 41664, 23, 'logs.pt')
ng20 = (200, 0.5, 80035, 20, 'gcn_20ng_full.pt', 61189, 18846, 11314,7532)
hidden_size,dropout,num_vertices,num_labels,path, num_words, num_docs, training_docs, test_docs = ng20


vocabulary = get_vocabulary('data/20ng-vocabulary.txt')
labels = get_labels('data/20ng-labels.txt')
test_corpus = get_data('data/test-20news-clean.txt', labels)
test_adj_matrix = load_sparse_tensor('data/20ng_full_adj_matrix_norm.pt')
train_corpus = get_data('data/train-20news-clean.txt', labels)

In [0]:
def matrices(path, adj):
  model = GCN(num_vertices, hidden_size, num_labels, dropout=dropout)
  model.load_state_dict(torch.load(path))
  model.eval()
  loss_fn = nn.CrossEntropyLoss()
  model.zero_grad()
  output = model(adj)
  print(output)
  loss = loss_fn(output[-test_docs:,:], test_corpus.labels())
  loss.retain_grad()
  loss.backward()
  weight_matrices = []
  gradients = []
  for n, p in model.named_parameters():
    print(n)
    weight_matrices.append(p.detach().numpy())
    gradients.append(p.grad.numpy())
    print(p.grad.shape)
  return weight_matrices, gradients, output

In [0]:
def grad_cam_avg(adj, weight_matrices, gradients):
  adj = adj.coalesce()
  V = sparse.csr_matrix((adj.values(), adj.indices()), shape=list(adj.size()))
  F_1 = scipy.special.expit(V.dot(weight_matrices[0]))
  F_2 = scipy.special.expit(V.dot(np.matmul(F_1, weight_matrices[1])))
  F = [F_1, F_2]
  grad_cams = []
  for l in range(2):
    grad_cam = []
    grad_cams.append(grad_cam)
    alphas = np.mean(gradients[l], axis = 0)
    for n in range(num_vertices):
      grad_cam.append(np.maximum(0, np.dot(alphas, F[l][n, :])))

  grad_cam_avgs = np.mean(grad_cams, axis=0)
  return grad_cams, grad_cam_avgs

In [49]:

#grad_cam for specific docs
indices = [14, 500]
gradient_test = []
for i in indices:
  single_corpus = Corpus([test_corpus.data[i]])
  adj = build_adj_matrix(single_corpus, vocabulary, num_docs, 1)
  adj = normalize_adj(adj)
  weight_matrices, gradients, output = matrices(path, adj)
  print(gradients)
  gradient_test.append(gradients)
  grad_cams, grad_cam_avgs = grad_cam_avg(adj, weight_matrices, gradients)
  print(test_corpus.data[-i].text)
  print(labels[test_corpus.data[-i].label])
  print(labels[np.argmax(output[-i,:].detach().numpy())])
  continue
  for name, grad_cam_vals in [('layer1', grad_cams[0]), ('layer2', grad_cams[1]), ('overall', grad_cam_avgs)]:
    print(name+":\n")
    top_words = np.asarray(grad_cam_vals).argsort()[-10:]
    for word_idx in top_words:
      if word_idx < num_words:
        print(vocabulary[word_idx])
      elif word_idx < num_words+training_docs:
        print(labels[train_corpus[word_idx-num_words].label])
      else:
        print(labels[train_corpus[word_idx-num_words-training_docs].label])

for i,j in zip(gradient_test[0], gradient_test[1]):
  for a,b in zip(i,j):
    for c,d in zip(a,b):
      if c != d:
        print(str(c) +  " " + str(d))
        print("OHOEHOEROEJRO")


Building word frequencies per doc



Building word frequencies per window



Calculating PMIs



Calculating TF-IDF



Identities


...


Building word frequencies per window



Calculating PMIs



Calculating TF-IDF



Identities



tensor([[-4.7116e+01,  0.0000e+00, -6.1410e+01,  ..., -7.3407e+01,
         -2.5230e+01, -5.3422e+01],
        [-3.3945e+00, -2.5673e+00, -3.8935e+00,  ..., -2.9769e+00,
         -2.8509e+00, -2.9981e+00],
        [-5.6530e+01, -4.6552e+01, -6.1059e+01,  ..., -6.7949e+01,
         -4.8304e+01, -2.3842e-07],
        ...,
        [-8.6512e+01, -8.4366e+01, -1.0313e+02,  ..., -9.8422e+01,
         -7.6604e+01,  0.0000e+00],
        [-8.0304e+01, -7.6062e+01, -1.0080e+02,  ..., -1.0354e+02,
         -7.2819e+01,  0.0000e+00],
        [-2.8939e+01, -4.3730e+01, -3.1337e+01,  ..., -1.8988e+01,
         -3.8788e+01, -3.8035e+00]], grad_fn=<LogSoftmaxBackward>)
layer1.weight
torch.Size([80035, 200])
layer2.weight
torch.Size([200, 20])
[array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  

In [19]:


total = 0
max_num = 10
for i in range(1, 7500):
  if test_corpus.data[-i].label == np.argmax(output[-i,:].detach().numpy()):
    print(i)
    total += 1
    if total == max_num:
      break

total = 0
for i in range(1, 7500):
  if test_corpus.data[-i].label == np.argmax(output[-i,:].detach().numpy()):
    total += 1
print(total/7500)

total = 0
for i in range(1, 7500):
  if test_corpus.data[-i].label != np.argmax(output[-i,:].detach().numpy()):
    print(i)
    total += 1
    if total == max_num:
      break

2
3
4
5
6
7
8
9
10
11
0.8578666666666667
1
14
15
20
22
24
29
30
32
35


In [20]:
def BE(adj, weight_matrices, gradients, output):
  adj = adj.coalesce()
  V = sparse.csr_matrix((adj.values(), adj.indices()), shape=list(adj.size()))
  F_1 = scipy.special.expit(V.dot(weight_matrices[0]))
  F_2 = scipy.special.expit(V.dot(np.matmul(F_1, weight_matrices[1])))
  F = [F_1, F_2]
  F_carrot = [V.dot(F_1), V.dot(F_2)]
  p_F3 = output


indices = [14, 500]
gradient_test = []
for i in indices:
  single_corpus = Corpus([test_corpus.data[i]])
  adj = build_adj_matrix(single_corpus, vocabulary, num_docs, 1)
  adj = normalize_adj(adj)
  weight_matrices, gradients, output = matrices(path, adj)
  BE(adj, weight_matrics, gradients, output)


torch.Size([80035, 20])
